In [7]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import seaborn as sns

In [9]:
METRICS_COLUMNS = ['psnr', 'ssim', 'fsim']
green_cmap = sns.light_palette('green', as_cmap=True)

In [10]:
df = pd.read_csv('data/super_res.csv')

# Only the final result
df = df[df['Step'] == df['num_steps'] - 1].copy()

# Combine dataset name
df.loc[:, 'dataset'] = df['dataset'] + '-' + df['split']
df = df.drop(columns=['split', 'Step', 'Unnamed: 0', 'num_steps'])

# Mean over the images
df = df.groupby(['model', 'dataset', 'sr_scale', 'sr_method', 'sr_antialias', 'algorithm']).mean()
df = df.drop(columns=['Image_Id'])
df = df.reset_index()

In [11]:
# List all datasets and models for usage in the next cell
print("Datasets:")
print('\n'.join(df['dataset'].unique()))
print("\nModels:")
print('\n'.join(df['model'].unique()))
print("\nSR Scales:")
print('\n'.join(df['sr_scale'].astype(str).unique()))
print("\nSR Methods:")
print('\n'.join(df['sr_method'].astype(str).unique()))
print("\nSR Antialiases:")
print('\n'.join(df['sr_antialias'].astype(str).unique()))
print("\nAlgorithms:")
print('\n'.join(df['algorithm'].unique()))

Datasets:
set14-test
set5-test

Models:
dae-v01-0.05
dae-v02-0.05
dae-v03-0.05
dae-v04-0.02
dae-v04-0.05
dae-v04-0.10
dae-v04-0.20
dae-v05-0.05
dae-v06-0.10
dae-v07-0.10
dae-v08-0.10
dae-v10-0.10
dae-v11-0.05
dae-v12-0.05
dae-v13-0.05
dae_0.04314
dngan-v01-0.05
dngan-v02-0.05
dngan-v03-0.05
dngan-v04-0.05
dngan-v05-0.05
dngan-v06-0.05
dngan-v07-0.05
dngan-v08-0.05
dngan-v09-0.05
dngan-v10-0.05
dngan-v11-0.05
dngan-v12-0.06
dngan-v13-0.20
dngan_resgen-vgg22_0.05
dngan_resgen-vgg22_ba_0.05
dngan_rrdb-vgg22_0.05
drunet_0.0_0.2
ffdgan-v01-0.0-0.2
ffdgan-v02-0.0-0.2
ffdgan-v03-0.0-0.2
ffdgan-v04-0.0-0.2
ffdgan-v05-0.0-0.2
ffdgan-v06-0.0-0.2
ffdnet-v01-0.0-0.2
ffdnet-v02-0.0-0.2
ffdnet-v03-0.0-0.2
ffdnet-v04-0.0-0.2

SR Scales:
2
3
4
5

SR Methods:
bicubic

SR Antialiases:
True

Algorithms:
dmsp


In [12]:
# Modify to see other results
params = {
    'dataset': 'set14-test',
    'sr_scale': 4,
    'sr_method': 'bicubic',
    'sr_antialias': True,
    'algorithm': 'dmsp',
    'model': 'dae-v04-0.10'
}

def _highlight(x):
    def rgb_for(v):
        if v == x.max():
            return (255, 255, 140)
        if v < 0.0:
            cv = (1 - (v / x.min())) * 255
            return (255, cv, cv)
        if v > 0:
            cv = (1 - (v / (x.max() / 20.0))) * 255
            return (cv, 255, cv)
        return (255, 255, 255)

    def text_color_for(v):
        r, g, b = rgb_for(v)
        lum = 0.2126*r + 0.7152*g + 0.0722*b
        if lum > 0.408:
            return 'black'
        else:
            return 'white'
    return [f'background-color: rgb{rgb_for(v)}; color: {text_color_for(v)}' for v in x]

def _filterDataset(dataset, sr_scale, sr_method, sr_antialias, algorithm):
    d = df[df['dataset'] == dataset]
    d = d[d['sr_scale'] == sr_scale]
    d = d[d['sr_method'] == sr_method]
    d = d[d['sr_antialias'] == sr_antialias]
    d = d[d['algorithm'] == algorithm]
    return d.drop(['dataset', 'sr_scale', 'sr_method', 'sr_antialias', 'algorithm'], axis=1)

def forDataset(**filters):
    d = _filterDataset(**filters)
    d = d.set_index('Model')
    return d.style.background_gradient(cmap=green_cmap, subset=METRICS_COLUMNS)

def forDatasetRelative(model, **filters):
    d = _filterDataset(**filters)

    # Compute relative values
    base_row = d[d['model'] == model]
    for metric in METRICS_COLUMNS:
        base_val = base_row[metric]
        d[metric] = d[metric] - base_val.values
        
    d = d.set_index('model')
    return d.style.apply(_highlight)

forDatasetRelative(**params)

,psnr,ssim,fsim
model,,,
dae-v01-0.05,-0.326925,-0.006564,-0.003430
dae-v02-0.05,-0.060450,-0.002366,-0.002300
dae-v03-0.05,-9.219175,-0.209285,-0.082084
dae-v04-0.02,-12.958033,-0.557756,-0.292940
dae-v04-0.05,-0.521982,-0.017454,-0.004420
dae-v04-0.10,0.000000,0.000000,0.000000
dae-v04-0.20,-0.036508,-0.003650,0.000575
dae-v05-0.05,-0.850442,-0.025582,-0.006021
dae-v06-0.10,-0.171695,-0.002514,-0.000772
